In [2]:
import pandas as pd
import plotly.express as px
import panel as pn
import tempfile
import sys
import re  # Import regex library

# Initialize Panel extension with required widgets
pn.extension('plotly', 'tabulator')

# ---------------------------
# Data Loading and Preparation
# ---------------------------
#custom_css = """
#
#.bk-root .tabulator .tabulator-header .tabulator-col .tabulator-col-title {
#    white-space: normal !important;
#    overflow-wrap: anywhere !important;
#    text-align: center;
#}
#"""

# pane with the custom CSS
#css_pane = pn.pane.HTML(custom_css, width=0, height=0, margin=0)

#pn.config.raw_css.append(custom_css)

# Define column definitions with widths for tabulator_2
columns_tabulator_1 = [
    {'title': 'COUNTY', 'field': 'COUNTY', 'width': 150},
    {'title': 'State Abbreviation', 'field': 'stusps', 'width': 100},
    {'title': 'Population 2022', 'field': 'pop2022', 'width': 150},
    {'title': '0 Bedroom FMR ($)', 'field': '0 Bedroom FMR ($)', 'width': 120},
    {'title': '1 Bedroom FMR ($)', 'field': '1 Bedroom FMR ($)', 'width': 120},
    {'title': '2 Bedroom FMR ($)', 'field': '2 Bedroom FMR ($)', 'width': 120},
    {'title': '3 Bedroom FMR ($)', 'field': '3 Bedroom FMR ($)', 'width': 120},
    {'title': '4 Bedroom FMR ($)', 'field': '4 Bedroom FMR ($)', 'width': 120},
    {'title': 'FIPS Code', 'field': 'fips5', 'width': 100},
]
columns_tabulator_2 = [
    {'title': 'COUNTY', 'field': 'COUNTY', 'width': 150},
    {'title': 'Median Income ($)', 'field': 'Median Income ($)', 'width': 200},
    # more columns to come
]
def load_fmr_data(filepath):
    try:
        df_fmr = pd.read_csv(filepath)
        print("Original FMR DataFrame Columns:", df_fmr.columns.tolist())  # Debugging

        # Removing trailing spaces from column names
        df_fmr.columns = df_fmr.columns.str.strip()

        # Extracting first 5 digits and ensure it's 5 digits with leading zeros
        df_fmr['fips5'] = df_fmr['fips'].astype(str).str[:5].str.zfill(5)
        df_fl_fips = df_fmr[df_fmr['stusps'] == 'FL'].copy()
        print("Filtered FL FIPS DataFrame Columns:", df_fl_fips.columns.tolist())  # Debugging

        # Ensuring 'fips5' is included in the mask
        mask_fmr = ['COUNTY', 'stusps', 'pop2022', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'fips5']
        df_fl_fmr = df_fl_fips[mask_fmr].rename(columns={
            'fmr_0': '0 Bedroom FMR ($)',
            'fmr_1': '1 Bedroom FMR ($)',
            'fmr_2': '2 Bedroom FMR ($)',
            'fmr_3': '3 Bedroom FMR ($)',
            'fmr_4': '4 Bedroom FMR ($)'
        }).copy()
        # print("Renamed FMR DataFrame Columns:", df_fl_fmr.columns.tolist())  # Debugging

        # Ensuring no leading/trailing spaces in 'COUNTY' column
        df_fl_fmr['COUNTY'] = df_fl_fmr['COUNTY'].str.strip()

        # Verify that '2 Bedroom FMR ($)' exists
        if '2 Bedroom FMR ($)' not in df_fl_fmr.columns:
            print("Error: '2 Bedroom FMR ($)' column not found after renaming.")
        else:
            print("'2 Bedroom FMR ($)' column successfully renamed and exists.")

        return df_fl_fips, df_fl_fmr
    except FileNotFoundError:
        pn.state.notifications.error("Error: 'FY25_FMRs.csv' not found.")
        sys.exit(1)
    except Exception as e:
        pn.state.notifications.error(f"Error loading FMR data: {e}")
        sys.exit(1)

def load_ami_data(filepath):
    try:
        df_ami = pd.read_csv(filepath)
        # print("Original AMI DataFrame Columns:", df_ami.columns.tolist())  # Debugging

        # Remove trailing spaces from column names
        df_ami.columns = df_ami.columns.str.strip()

        mask_ami_median = ['COUNTY', 'Median Income ($)']
        mask_ami_housing = ['COUNTY', 'AMI Category','1 Person Limit ($)','2 Person Limit ($)','3 Person Limit ($)','4 Person Limit ($)']
        mask_ami_rental = ['COUNTY', 'AMI Category','0 Bedroom Limit ($)','1 Bedroom Limit ($)','2 Bedroom Limit ($)','3 Bedroom Limit ($)','4 Bedroom Limit ($)']

        df_ami_median = df_ami[mask_ami_median].copy()
        df_ami_housing = df_ami[mask_ami_housing].copy()
        df_ami_rental = df_ami[mask_ami_rental].copy()

        # Optional: Ensure no leading/trailing spaces in 'COUNTY' column
        df_ami_median['COUNTY'] = df_ami_median['COUNTY'].str.strip()
        df_ami_housing['COUNTY'] = df_ami_housing['COUNTY'].str.strip()
        df_ami_rental['COUNTY'] = df_ami_rental['COUNTY'].str.strip()

        #print("AMI Median DataFrame Columns:", df_ami_median.columns.tolist())  # Debugging
        #print("AMI Housing DataFrame Columns:", df_ami_housing.columns.tolist())  # Debugging
        #print("AMI Rental DataFrame Columns:", df_ami_rental.columns.tolist())  # Debugging

        return df_ami_median, df_ami_housing, df_ami_rental
    except FileNotFoundError:
        pn.state.notifications.error("Error: 'fl_ami_income_limit.csv' not found.")
        sys.exit(1)
    except Exception as e:
        pn.state.notifications.error(f"Error loading AMI data: {e}")
        sys.exit(1)

def load_jobs_data(filepath):
    try:
        df_jobs = pd.read_csv(filepath)
        #print("Original Jobs DataFrame Columns:", df_jobs.columns.tolist())  # Debugging

        # Remove trailing spaces from column names
        df_jobs.columns = df_jobs.columns.str.strip()

        mask_ami_jobs = [
            'COUNTY', 'Geography', 'Occupation', 'Median Hourly Wage (2023 $)', 
            'Maximum Affordable Rent (30% of Income)', '2023 HUD 2BR Fair Market Rent',
            '% Income Needed for 2 BR, Median Wage Worker', '# of Workers in 2022',
            'Annual Wage as a % AMI for a Family of 3'
        ]
        df_ami_jobs = df_jobs[mask_ami_jobs].copy()

        # Clean up 'COUNTY' column
        df_ami_jobs['COUNTY'] = df_ami_jobs['COUNTY'].str.strip()

        # Split 'COUNTY' into a list of counties
        df_ami_jobs['COUNTY_LIST'] = df_ami_jobs['COUNTY'].str.split(',')

        # Remove leading/trailing spaces from each county in the list
        df_ami_jobs['COUNTY_LIST'] = df_ami_jobs['COUNTY_LIST'].apply(lambda x: [c.strip() for c in x])

        # Explode the DataFrame so each county has its own row
        df_ami_jobs = df_ami_jobs.explode('COUNTY_LIST')

        # Now, 'COUNTY_LIST' contains single county names
        # Rename 'COUNTY_LIST' back to 'COUNTY' for consistency
        df_ami_jobs['COUNTY'] = df_ami_jobs['COUNTY_LIST']
        df_ami_jobs.drop(columns=['COUNTY_LIST'], inplace=True)

       # print("Filtered Jobs DataFrame Columns:", df_ami_jobs.columns.tolist())  # Debugging
       # print("Sample Jobs DataFrame after exploding 'COUNTY':")
       # print(df_ami_jobs.head())  # Debugging

        return df_ami_jobs
    except FileNotFoundError:
        pn.state.notifications.error("Error: 'fl_ami_occupation_wage.csv' not found.")
        sys.exit(1)
    except Exception as e:
        pn.state.notifications.error(f"Error loading Jobs data: {e}")
        sys.exit(1)

# Load all datasets
df_fl_fips, df_fl_fmr = load_fmr_data('FY25_FMRs.csv')
df_ami_median, df_ami_housing, df_ami_rental = load_ami_data('fl_ami_income_limit.csv')
df_ami_jobs = load_jobs_data('fl_ami_occupation_wage.csv')

# ---------------------------
# Define Data Sources and Reports
# ---------------------------

# FMR Data
datasource_fmr = 'U.S. Department of Housing and Urban Development (HUD)'
report_name_fmr = 'Fair Market Rents 2025 County Level Data'
datasource_url_fmr = 'https://www.huduser.gov/portal/datasets/fmr.html#data_2025'

# AMI Income Limits Data
datasource_ami = 'Florida Housing Finance Corporation'
report_name_ami = 'SHIP and HHRP - 2024 Combined Income and Rent Limits (Eff. 4-1-2024)'
datasource_url_ami = 'https://www.floridahousing.org/owners-and-managers/compliance/income-limits'

# Wage and Occupation Data
datasource_occupation = 'Florida Housing Data Clearinghouse'
report_name_occupation = 'Wage and Rent Comparison by Occupation, 2023'
datasource_url_occupation = 'http://flhousingdata.shimberg.ufl.edu/affordability/results?nid=100000'

# ---------------------------
# Widget Definition
# ---------------------------

# County selection widget
county_names = sorted(df_fl_fmr['COUNTY'].unique().tolist())

select_county = pn.widgets.Select(
    name='Select County',
    options=['All Counties'] + county_names,
    value='All Counties',
    sizing_mode='stretch_width'
)

# Loading spinner (optional, since updates are reactive and likely fast)
loading_spinner = pn.indicators.LoadingSpinner(value=False, width=50, height=50, visible=False)

# ---------------------------
# Create Tabulator Tables
# ---------------------------

# Function to create table layouts with download buttons
def create_table_layout(report_name, download_button, datasource_url, tabulator):
    return pn.Column(
        pn.pane.Markdown(f"## {report_name}"),
        pn.Row(download_button, pn.pane.Markdown(f"[DataSource]({datasource_url})")),
        tabulator,
        sizing_mode='stretch_width'
    )

# Create Tabulator tables for each dataset
tabulator_1 = pn.widgets.Tabulator(
    df_fl_fmr, 
    height=150,  # Increased height to display more rows
    sizing_mode='stretch_width', 
    name="FMR Data",
    visible=False,  # Initially hidden
    pagination='local',  # Enable local pagination
    page_size=50,  # Set 50 rows per page
    selectable='checkbox',  # Optional: Enable row selection
    sortable=True           # Optional: Enable column sorting
)

tabulator_2 = pn.widgets.Tabulator(
    df_ami_median, 
    height=150, 
    sizing_mode='stretch_width', 
    name="AMI Median Income",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

tabulator_3 = pn.widgets.Tabulator(
    df_ami_housing, 
    height=300, 
    sizing_mode='stretch_width', 
    name="AMI Housing Limits",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

tabulator_4 = pn.widgets.Tabulator(
    df_ami_rental, 
    height=300, 
    sizing_mode='stretch_width', 
    name="AMI Rental Limits",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

tabulator_5 = pn.widgets.Tabulator(
    df_ami_jobs, 
    height=600, 
    sizing_mode='stretch_width', 
    name="Wage and Occupation",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

# Create download buttons for all tables
def create_csv_button(df, filename):
    def csv_callback():
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
        df.to_csv(temp_file.name, index=False)
        return temp_file.name
    return pn.widgets.FileDownload(callback=csv_callback, filename=filename, button_type='primary')

download_button_1 = create_csv_button(df_fl_fmr, "fmr.csv")
download_button_2 = create_csv_button(df_ami_median, "ami_median.csv")
download_button_3 = create_csv_button(df_ami_housing, "ami_housing.csv")
download_button_4 = create_csv_button(df_ami_rental, "ami_rental.csv")
download_button_5 = create_csv_button(df_ami_jobs, "ami_jobs.csv")

# Create individual table layouts
table_1_layout = create_table_layout(report_name_fmr, download_button_1, datasource_url_fmr, tabulator_1)
table_2_layout = create_table_layout(report_name_ami, download_button_2, datasource_url_ami, tabulator_2)
table_3_layout = pn.Column(
    pn.pane.Markdown(f"## Housing Limits by AMI"),
    pn.Row(download_button_3),
    tabulator_3,
    sizing_mode='stretch_width'
)
table_4_layout = pn.Column(
    pn.pane.Markdown("## Rental Limits by AMI"),
    pn.Row(download_button_4),
    tabulator_4,
    sizing_mode='stretch_width'
)
table_5_layout = create_table_layout(report_name_occupation, download_button_5, datasource_url_occupation, tabulator_5)

# ---------------------------
# Choropleth Map Function
# ---------------------------

def create_map(selected_county):
    """
    Creates a Choropleth map based on the selected county.
    """
    if selected_county == 'All Counties':
        df = df_fl_fmr.copy()
        title = 'Florida Counties Map - Population 2022'
    else:
        df = df_fl_fmr[df_fl_fmr['COUNTY'] == selected_county].copy()
        title = f'Florida County Map - {selected_county} (Population 2022)'

    # Create the Choropleth map
    fig = px.choropleth(
        df,
        geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json',
        locations='fips5',
        color='pop2022',  # Use numerical column for color scaling
        scope='usa',
        labels={'COUNTY': 'FL Counties'},
        title=title,
        hover_data=['COUNTY', 'pop2022']  # Ensure these columns exist
    )

    # Update the map's geographical properties
    fig.update_geos(fitbounds="locations", visible=False)

    return fig

# ---------------------------
# Panel Widgets and Plotly Pane
# ---------------------------

# Plotly Pane to display the Choropleth map
initial_fig = create_map('All Counties')
plotly_pane = pn.pane.Plotly(initial_fig, sizing_mode='stretch_width', height=400)

# ---------------------------
# Reactive Update Function
# ---------------------------

def update_dashboard(event):
    selected_county = event.new.strip()  # Remove any leading/trailing spaces
    loading_spinner.visible = True  # Show loading spinner
    # print(f"---\nSelected County: {selected_county}")  # Debugging

    # Filter data based on selected county
    if selected_county == 'All Counties':
        filtered_fmr_df = df_fl_fmr.copy()
        filtered_ami_median_df = df_ami_median.copy()
        filtered_ami_housing_df = df_ami_housing.copy()
        filtered_ami_rental_df = df_ami_rental.copy()
        filtered_ami_jobs_df = df_ami_jobs.copy()
        print("Selected 'All Counties'. Showing all data.")
    else:
        # Filter DataFrames based on 'COUNTY' column
        filtered_fmr_df = df_fl_fmr[df_fl_fmr['COUNTY'] == selected_county].copy()
        filtered_ami_median_df = df_ami_median[df_ami_median['COUNTY'] == selected_county].copy()
        filtered_ami_housing_df = df_ami_housing[df_ami_housing['COUNTY'] == selected_county].copy()
        filtered_ami_rental_df = df_ami_rental[df_ami_rental['COUNTY'] == selected_county].copy()
        filtered_ami_jobs_df = df_ami_jobs[df_ami_jobs['COUNTY'] == selected_county].copy()

       # print(f"Filtered data for county: {selected_county}")
       # print(f"Filtered FMR DataFrame Rows: {len(filtered_fmr_df)}")
       # print(f"Filtered AMI Median DataFrame Rows: {len(filtered_ami_median_df)}")
       # print(f"Filtered AMI Housing DataFrame Rows: {len(filtered_ami_housing_df)}")
       # print(f"Filtered AMI Rental DataFrame Rows: {len(filtered_ami_rental_df)}")
       # print(f"Filtered AMI Jobs DataFrame Rows: {len(filtered_ami_jobs_df)}")

    # Remove duplicates
    filtered_fmr_df = filtered_fmr_df.drop_duplicates()
    filtered_ami_median_df = filtered_ami_median_df.drop_duplicates()
    filtered_ami_housing_df = filtered_ami_housing_df.drop_duplicates()
    filtered_ami_rental_df = filtered_ami_rental_df.drop_duplicates()
    filtered_ami_jobs_df = filtered_ami_jobs_df.drop_duplicates()    

    # Update the Choropleth map
    if selected_county == 'All Counties':
        data_for_map = df_fl_fmr.copy()
        title = 'Florida Counties Map - Population 2022'
    else:
        data_for_map = filtered_fmr_df.copy()
        title = f'Florida County Map - {selected_county} (Population 2022)'

    # Ensure 'fips5' is string type and zero-padded
    data_for_map['fips5'] = data_for_map['fips5'].astype(str).str.zfill(5)

    # Debugging: Print a sample of the data used for the map
    # print("Sample Data for Map:")
    # print(data_for_map[['COUNTY', 'fips5', 'pop2022']].head())

    # Debugging: Print DataFrame columns to ensure 'pop2022' exists
    # print("Columns in data_for_map:", data_for_map.columns.tolist())

    try:
        fig = px.choropleth(
            data_for_map,
            geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json',
            locations='fips5',
            color='pop2022',  # Use numerical column for color scaling
            scope='usa',
            labels={'COUNTY': 'FL Counties'},
            title=title,
            hover_data=['COUNTY', 'pop2022']  # Ensure these columns exist
        )
    except Exception as e:
        pn.state.notifications.error(f"Error creating Choropleth map: {e}")
        print(f"Error creating Choropleth map: {e}")
        loading_spinner.visible = False
        return

    fig.update_geos(fitbounds="locations", visible=False)

    # Update the Plotly pane
    plotly_pane.object = fig
    print("Choropleth map updated successfully.")

    # Update Tabulator tables
    tabulator_1.value = filtered_fmr_df
    tabulator_2.value = filtered_ami_median_df
    tabulator_3.value = filtered_ami_housing_df
    tabulator_4.value = filtered_ami_rental_df
    tabulator_5.value = filtered_ami_jobs_df
    print("Tabulator tables updated successfully.")

    # Update visibility of Tables based on selection
    if selected_county == 'All Counties':
        tabulator_1.visible = False
        tabulator_2.visible = False
        tabulator_3.visible = False
        tabulator_4.visible = False
        tabulator_5.visible = False
        print("All tables hidden.")
    else:
        tabulator_1.visible = True
        tabulator_2.visible = True
        tabulator_3.visible = True
        tabulator_4.visible = True
        tabulator_5.visible = True
        print("Tables made visible.")

    loading_spinner.visible = False  # Hide loading spinner
    print("---\n")  # Separator for readability

# ---------------------------
# Attaching the Reactive Function as a Callback
# ---------------------------

# Attaching update_dashboard function to the select_county widget
select_county.param.watch(update_dashboard, 'value')

# ---------------------------
# Dashboard Layout with Tabs
# ---------------------------

# Tab 1: Map and County Selector
tab1_content = pn.Column(
    pn.pane.Markdown("## County Map"),
    pn.pane.Markdown("""
        <div style="font-size: 18px;">
        Select a specific County from the list to see downloadable tabular ressults in the other tabs. 
        More to come.
        </div>
    """),
    pn.Row(
        pn.Column(
            pn.pane.Markdown("### Select County"),
            select_county,
            loading_spinner,
            sizing_mode='stretch_width',
            height=100
        ),
        plotly_pane,
        sizing_mode='stretch_both'
    )
)

# Create Tabs
tabs = pn.Tabs(
    ("Map", tab1_content),
    ("Fair Market Rents", table_1_layout),
    ("AMI Median Income", table_2_layout),
    ("Housing Limits by AMI", table_3_layout),
    ("Rental Limits by AMI", table_4_layout),
    ("Wage and Occupation", table_5_layout),
    sizing_mode='stretch_both'
)


# ---------------------------
# Display the Dashboard with Tabs
# ---------------------------

#tabs
# To serve as a standalone application, uncomment the line below:
pn.serve(tabs, threaded=True)


Original FMR DataFrame Columns: ['stusps', 'state', 'hud_area_code', 'COUNTY', 'county_town_name', 'metro', 'hud_area_name', 'fips', 'pop2022', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'DataSource ', 'Report Name', 'URL']
Filtered FL FIPS DataFrame Columns: ['stusps', 'state', 'hud_area_code', 'COUNTY', 'county_town_name', 'metro', 'hud_area_name', 'fips', 'pop2022', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'DataSource', 'Report Name', 'URL', 'fips5']
'2 Bedroom FMR ($)' column successfully renamed and exists.


<StoppableThread(Thread-6 (get_server), started 47200)>